In [1]:
import lightkurve as lk



f:\Python\Python312\Lib\site-packages\lightkurve\prf\__init__.py:7: UserWarning: Warning: the tpfmodel submodule is not available without oktopus installed, which requires a current version of autograd. See #1452 for details.
  warnings.warn(


In [ ]:
import lightkurve as lk

name = "TIC 317951248"
lc = lk.search_targetpixelfile(name)[1].download()

f:\Python\Python312\Lib\site-packages\lightkurve\prf\__init__.py:7: UserWarning: Warning: the tpfmodel submodule is not available without oktopus installed, which requires a current version of autograd. See #1452 for details.
  warnings.warn(


In [3]:
lc.animate()

In [ ]:
name = "TIC 425220920"
lc = lk.search_targetpixelfile(name)[1].download()

In [2]:
lc = lk.search_lightcurvefile("TIC 425220920", mission="TESS")[0].download(download_dir="cache")


C:\Users\hayka\AppData\Local\Temp\ipykernel_26056\2807687663.py:1: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  lc = lk.search_lightcurvefile("TIC 425220920", mission="TESS")[0].download(download_dir="cache")


In [7]:
lc = lk.search_lightcurvefile("TIC 177126182", mission="TESS")[1].download(download_dir="cache")


C:\Users\hayka\AppData\Local\Temp\ipykernel_26056\1021825229.py:1: LightkurveDeprecationWarning: The search_lightcurvefile function is deprecated and may be removed in a future version.
        Use search_lightcurve() instead.
  lc = lk.search_lightcurvefile("TIC 177126182", mission="TESS")[1].download(download_dir="cache")


In [8]:
lc.animate()

AttributeError: object has no attribute animate

In [5]:
from astroquery.mast import Observations

obs = Observations.query_object("TIC 425220920", radius="0s")
products = Observations.get_product_list(obs)
filtered = Observations.filter_products(products, productSubGroupDescription="LC")
Observations.download_products(filtered, mrp_only=False, download_dir="cache")


If you are looking for TESS image data for a specific target, please use TESScut at https://mast.stsci.edu/tesscut/.
If you need a TESS image for an entire field, please see our dedicated page for downloading larger quantities of TESS data at 
https://archive.stsci.edu/tess/. Data products will not be fetched for the following observations IDs: 
tess-s0007-1-4
tess-s0033-1-3
tess-s0087-1-3 [astroquery.mast.observations]
Please see our dedicated page for downloading larger quantities of TICA data: https://archive.stsci.edu/hlsp/tica.
Data products will not be fetched for the following observation IDs: 
hlsp_tica_s0033-cam1-ccd3
hlsp_tica_s0087-cam1-ccd3 [astroquery.mast.observations]


In [9]:
import requests

url = "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:TESS/product/tess2020203142519-s0014-0000000425220920-0188-s_lc.fits"
r = requests.get(url)
with open("tic425220920_lc.fits", "wb") as f:
    f.write(r.content)


In [13]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import os

# Dosya kontrolü
fits_filename = "tic425220920_lc.fits"

print(f"🔍 Checking file: {fits_filename}")
if os.path.exists(fits_filename):
    file_size = os.path.getsize(fits_filename)
    print(f"📁 File exists - Size: {file_size} bytes")
    
    if file_size < 1000:  # Çok küçükse muhtemelen hatalı
        print("⚠️ File seems too small, might be corrupted")
        print("🔄 Re-downloading the file...")
        
        # Dosyayı yeniden indir
        import requests
        url = "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:TESS/product/tess2020203142519-s0014-0000000425220920-0188-s_lc.fits"
        try:
            r = requests.get(url, timeout=30)
            r.raise_for_status()
            with open(fits_filename, "wb") as f:
                f.write(r.content)
            print(f"✅ File re-downloaded - New size: {len(r.content)} bytes")
        except Exception as e:
            print(f"❌ Download failed: {e}")
            
else:
    print("❌ File not found!")
    print("🔄 Downloading the file...")
    
    # Dosyayı indir
    import requests
    url = "https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:TESS/product/tess2020203142519-s0014-0000000425220920-0188-s_lc.fits"
    try:
        r = requests.get(url, timeout=30)
        r.raise_for_status()
        with open(fits_filename, "wb") as f:
            f.write(r.content)
        print(f"✅ File downloaded - Size: {len(r.content)} bytes")
    except Exception as e:
        print(f"❌ Download failed: {e}")

# FITS dosyasını aç
try:
    print(f"\n📂 Opening FITS file...")
    hdul = fits.open(fits_filename)
    print("✅ FITS file opened successfully!")
    hdul.info()
    
    # Genelde ışık eğrisi 1. extension'da (index=1)
    data = hdul[1].data  
    
    time = data['TIME']          # gün cinsinden (BJD - 2457000)
    flux = data['PDCSAP_FLUX']   # normalize edilmiş akı
    flux_err = data['PDCSAP_FLUX_ERR']
    
    # Basit temizlik: NaN değerleri çıkar
    mask = ~np.isnan(time) & ~np.isnan(flux)
    time, flux, flux_err = time[mask], flux[mask], flux_err[mask]
    
    print(f"\n📊 Data points: {len(time)}")
    print(f"📅 Time range: {time.min():.2f} - {time.max():.2f} days")
    print(f"💡 Flux range: {flux.min():.0f} - {flux.max():.0f} e-/s")
    
    # Geliştirilmiş grafik
    plt.figure(figsize=(12, 6))
    plt.scatter(time, flux, s=0.5, color="darkblue", alpha=0.7)
    plt.xlabel("Time (BJD - 2457000)", fontsize=12)
    plt.ylabel("PDCSAP Flux (e-/s)", fontsize=12)
    plt.title("TESS Light Curve - TIC 425220920", fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Basit istatistikler
    flux_median = np.median(flux)
    flux_std = np.std(flux)
    print(f"\n📈 Median flux: {flux_median:.0f} e-/s")
    print(f"📊 Standard deviation: {flux_std:.0f} e-/s")
    print(f"🔍 Relative variability: {(flux_std/flux_median)*100:.3f}%")
    
    hdul.close()
    
except Exception as e:
    print(f"❌ Error opening FITS file: {e}")
    print("🔧 Trying with ignore_missing_simple=True...")
    try:
        hdul = fits.open(fits_filename, ignore_missing_simple=True)
        print("✅ FITS file opened with ignore_missing_simple=True!")
        hdul.info()
        hdul.close()
    except Exception as e2:
        print(f"❌ Still failed: {e2}")
        print("📝 File might be corrupted or not a valid FITS file")

🔍 Checking file: tic425220920_lc.fits
📁 File exists - Size: 22 bytes
⚠️ File seems too small, might be corrupted
🔄 Re-downloading the file...
❌ Download failed: 404 Client Error: Not Found for url: https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:TESS/product/tess2020203142519-s0014-0000000425220920-0188-s_lc.fits

📂 Opening FITS file...
❌ Error opening FITS file: No SIMPLE card found, this file does not appear to be a valid FITS file. If this is really a FITS file, try with ignore_missing_simple=True
🔧 Trying with ignore_missing_simple=True...
❌ Still failed: Empty or corrupt FITS file
📝 File might be corrupted or not a valid FITS file
❌ Download failed: 404 Client Error: Not Found for url: https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:TESS/product/tess2020203142519-s0014-0000000425220920-0188-s_lc.fits

📂 Opening FITS file...
❌ Error opening FITS file: No SIMPLE card found, this file does not appear to be a valid FITS file. If this is really a FITS file, try with ignor

        Use textwrap.indent() instead. [astropy.io.fits.hdu.hdulist]
    Header size is not multiple of 2880: 22
There may be extra bytes after the last HDU or the file is corrupted. [astropy.io.fits.hdu.hdulist]
